In [1]:
import sys
sys.path.insert(0, '../../') 
from project.data.datasets.ARG_KP_2021.processor import load_dataset
import pandas as pd

### load all arguments with quality score and ground truth kps 

In [40]:
#train_df = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\train_complete.csv", index_col=None)
#dev_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\dev_complete.csv", index_col=None)
test_df =  pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\test_complete.csv", index_col=None)

In [3]:
print(len(test_df))
test_df.head(2)

3923


,idx,arg_id,argument,topic,stance,MACE-P,key_point_id,key_point,label,label_strict,label_relaxed,text,sentence1,sentence2
0,0,arg_0_0,Routine child vaccinations isn t mandatory sin...,Routine child vaccinations should be mandatory,-1,0.714195,kp_0_0,Routine child vaccinations or their side effe...,0.0,0.0,0.0,Routine child vaccinations isn t mandatory sin...,[CLS] Routine child vaccinations isn t mandato...,[CLS] Routine child vaccinations or their sid...
1,1,arg_0_0,Routine child vaccinations isn t mandatory sin...,Routine child vaccinations should be mandatory,-1,0.714195,kp_0_1,Mandatory vaccination contradicts basic rights,0.0,0.0,0.0,Routine child vaccinations isn t mandatory sin...,[CLS] Routine child vaccinations isn t mandato...,[CLS] Mandatory vaccination contradicts basic ...


In [78]:
# import the potential kps, was prepare in a separate noteook (prepare_potential_kps.ipnyb)
# try with clustered filtered sentences
all_pot_sent_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\ARG_KP_2021\candidates_clustering_test.csv', index_col=None)

In [79]:
all_pot_sent_df

,sentence,cluster_score,aspects_id,topic_size,topic,stance,q_score,word_count
0,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,0.933753,0,80,Routine child vaccinations should be mandatory,-1,0.871068,17
1,Children should not be vaccinated because they can have serious side effects,0.930660,0,80,Routine child vaccinations should be mandatory,-1,0.865730,12
2,child vaccinations shouldn't be mandatory because the vaccine can have side effects on children,0.927722,0,80,Routine child vaccinations should be mandatory,-1,0.878956,14
3,child vaccinations should not be mandatory to avoid any bad effects on them,0.902686,0,80,Routine child vaccinations should be mandatory,-1,0.773095,13
4,Child vaccination should not be mandatory as there are often side effects from them,0.896708,0,80,Routine child vaccinations should be mandatory,-1,0.842266,14
...,...,...,...,...,...,...,...,...
92,we all have the american dream. it is the best country in the world without a doubt,0.827796,2,20,The USA is a good country to live in,1,0.717036,17
93,America is a free nation where working hard you can live the American dream,0.822749,2,20,The USA is a good country to live in,1,0.714445,14
94,Freedom of expression and the American dream is the most desired,0.798234,2,20,The USA is a good country to live in,1,0.713084,11
95,"Fulfill the American dream of equality, freedom of expression and quality of life.",0.716145,2,20,The USA is a good country to live in,1,0.696301,13


In [41]:
# to be able to compare, we take only arguments with positive single label
test_ds_labeled = test_df[test_df['label'] == 1]
test_ds_labeled = test_ds_labeled.sample(frac=1).drop_duplicates(subset='arg_id')

### create cross args dataset

Get every possible pair to get matching score from best model. 

In [5]:
# all_pot_sent_df has potential kps sentences and test_ds_labeled has the arguments we want to generate kps for them

def get_paired_args(arguments_df, potential_kps, topic, stance):
    args_ids = []
    args = []
    kps = []
    q_scores = []
    
    filtered_arguments_df = arguments_df[arguments_df['topic'] == topic]
    filtered_potential_kps_df = potential_kps[potential_kps['topic'] == topic]
    
    filtered_arguments_df = filtered_arguments_df[filtered_arguments_df['stance'] == stance]
    filtered_potential_kps_df = filtered_potential_kps_df[filtered_potential_kps_df['stance'] == stance]
    
    unique_args_id = filtered_arguments_df['arg_id'].unique()
    
    for arg_id in unique_args_id:
        arg_row = filtered_arguments_df[filtered_arguments_df["arg_id"] == arg_id].iloc[0]
        for  index, s_row in filtered_potential_kps_df.iterrows():
            if arg_row["argument"] == s_row["sentence"]:
                continue
            args_ids.append(arg_id)
            args.append(arg_row["argument"])
            kps.append(s_row["sentence"])
            q_scores.append(s_row["q_score"])
    
    res_df = pd.DataFrame({
        "arg_id": args_ids,
        "argument": args,
        "key_point": kps,
        "kp_q_score": q_scores
    })
    
    res_df['topic'] = topic
    res_df['stance'] = stance
    return res_df

topics = test_ds_labeled['topic'].unique()
stances = [-1, 1]

cross_pairs_df = None
for topic in topics:
    for stance in stances:
        if cross_pairs_df is None:
            cross_pairs_df = get_paired_args(test_ds_labeled, all_pot_sent_df,  topic, stance)
        else:
            cross_pairs_df = pd.concat([cross_pairs_df, get_paired_args(test_ds_labeled, all_pot_sent_df, topic, stance)])

In [6]:
cross_pairs_df.head(2)

,arg_id,argument,key_point,kp_q_score,topic,stance
0,arg_1_41,Social media platforms should not be regulated...,Social media platform should not be regulated ...,0.966574,Social media platforms should be regulated by ...,-1
1,arg_1_41,Social media platforms should not be regulated...,Social media platforms should not be regulated...,0.848147,Social media platforms should be regulated by ...,-1


In [7]:
len(cross_pairs_df)

11578

In [8]:
cross_pairs_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_cluster_df_basic.csv', index=False)

### load best model 

Here we load best matcher model and score the pairs

In [4]:
import pandas as pd
test_cross_pairs_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_cluster_df_basic.csv', index_col=None)

In [5]:
len(test_cross_pairs_df)

11578

In [6]:
test_cross_pairs_df['idx'] = test_cross_pairs_df.index
test_cross_pairs_df.head(2)

,arg_id,argument,key_point,kp_q_score,topic,stance,idx
0,arg_1_41,Social media platforms should not be regulated...,Social media platform should not be regulated ...,0.966574,Social media platforms should be regulated by ...,-1,0
1,arg_1_41,Social media platforms should not be regulated...,Social media platforms should not be regulated...,0.848147,Social media platforms should be regulated by ...,-1,1


In [12]:
# process data to work with the classifier
from project.data.datasets.ARG_KP_2021.processor import preprocessing

test_cross_pairs_df['argument'] = test_cross_pairs_df['argument'].apply(lambda x: preprocessing(x))
test_cross_pairs_df['key_point'] = test_cross_pairs_df['key_point'].apply(lambda x: preprocessing(x))
test_cross_pairs_df['topic'] = test_cross_pairs_df['topic'].apply(lambda x: preprocessing(x))

test_cross_pairs_df['sentence1'] = '[CLS] ' + test_cross_pairs_df['argument'] + ' [SEP] ' + test_cross_pairs_df['topic'] + ' [SEP]'
test_cross_pairs_df['sentence2'] = '[CLS] ' + test_cross_pairs_df['key_point'] + ' [SEP] ' + test_cross_pairs_df['topic'] + ' [SEP]'

In [13]:
test_cross_pairs_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_clean_df_cluster.csv', index=False)

In [7]:
# load best model
import torch
torch.cuda.empty_cache()
from project.models.baselines.classification_model import ClassificationModel
from project.models.common.nlp_configs import HFTransformerDataConfig
from transformers import AutoTokenizer
from project.data.nlp_data_module import NLPDataModule

test_data_path = r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_clean_df_cluster.csv'
model_checkpoint_path = r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\models\good-models\epoch=1-step=807.ckpt"
model = ClassificationModel.load_from_checkpoint(checkpoint_path=model_checkpoint_path)
data_cfg = HFTransformerDataConfig(**{
    "batch_size": 32,
    "num_workers": 4,
    "train_file": test_data_path,
    "test_file": test_data_path
})
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model.model.config._name_or_path, use_fast=True)
datamodule = NLPDataModule(cfg=data_cfg, tokenizer=tokenizer)
datamodule.setup("test")
test_dataset = datamodule.ds["test"]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration default-e3fe4787df4f5919


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\Bilal\.cache\huggingface\datasets\csv\default-e3fe4787df4f5919\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [8]:
test_dataset.column_names

['arg_id',
 'argument',
 'attention_mask',
 'attention_mask_sent1',
 'attention_mask_sent2',
 'idx',
 'input_ids',
 'input_ids_sent1',
 'input_ids_sent2',
 'key_point',
 'kp_q_score',
 'sentence1',
 'sentence2',
 'stance',
 'topic']

In [9]:
# score match score for every possible pair
BATCH_SIZE = 32

test_ds_df = test_dataset.to_pandas()
test_ds_df.set_index('idx')
test_ds_df['score'] = -1

model.to('cuda')

def add_preds(examples):
    similarity, _, output = model._forward(examples, predict_with_gpu=True)
    for i, idx in enumerate(examples["idx"]):
        if output is not None:
            score = output[i][1].item()
        else:
            score = similarity[i].item()
        test_ds_df.iat[idx.item(), test_ds_df.columns.get_loc('score')] = score

test_dataset.map(lambda examples: add_preds(examples), batched=True, batch_size=BATCH_SIZE)



  0%|          | 0/362 [00:00<?, ?ba/s]

Dataset({
    features: ['arg_id', 'argument', 'attention_mask', 'attention_mask_sent1', 'attention_mask_sent2', 'idx', 'input_ids', 'input_ids_sent1', 'input_ids_sent2', 'key_point', 'kp_q_score', 'sentence1', 'sentence2', 'stance', 'topic'],
    num_rows: 11578
})

In [10]:
test_ds_df.head(2)
test_ds_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_with_match_score_cluster.csv', index=False)

In [11]:
import pandas as pd
test_ds_df = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_cross_pairs_with_match_score_cluster.csv', index_col=None)

In [12]:
print(len(test_ds_df))
test_ds_df.head(1)

11578


,arg_id,argument,attention_mask,attention_mask_sent1,attention_mask_sent2,idx,input_ids,input_ids_sent1,input_ids_sent2,key_point,kp_q_score,sentence1,sentence2,stance,topic,score
0,arg_1_41,Social media platforms should not be regulated...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,0,[ 0 10975 7454 104 742 3574 433 48...,[ 0 10975 7454 104 742 3574 433 48...,[ 0 10975 7454 104 742 3574 433 17...,Social media platform should not be regulated ...,0.966574,[CLS] Social media platforms should not be reg...,[CLS] Social media platform should not be regu...,-1,Social media platforms should be regulated by ...,0.818392


In [13]:
test_ds_df[['sentence1', 'sentence2', 'stance', 'topic', 'score']][test_ds_df['score']> 0.5].sort_values(by='score', ascending=True).head(1)

,sentence1,sentence2,stance,topic,score
8371,[CLS] to keep schools safe children must be va...,[CLS] Vaccination is a safe and effective way ...,1,Routine child vaccinations should be mandatory,0.500058


In [14]:
# get top kps based on their number of matches (based on a threshold)
threshold = 0.5
result_df = test_ds_df[test_ds_df['score'] > threshold]
# choose best possible kp considering quality + match + rule-based
# first for every arg_id, get top 20 matches
# result_df = test_ds_df.groupby(by="arg_id").apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="score", ascending=False).head(1)).reset_index(drop=True)
# add combined average score quality+match/2 
# result_df['combined_score'] = (result_df['kp_q_score'] + result_df['score'] )/ 2
# second take the arg with top combined score from the 20 matches
# result_df = result_df.groupby(by="arg_id").apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by="combined_score", ascending=False).head(1)).reset_index(drop=True)

In [15]:
len(result_df)

4828

In [16]:
print(len(result_df))
result_df.head(1)

4828


,arg_id,argument,attention_mask,attention_mask_sent1,attention_mask_sent2,idx,input_ids,input_ids_sent1,input_ids_sent2,key_point,kp_q_score,sentence1,sentence2,stance,topic,score
0,arg_1_41,Social media platforms should not be regulated...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,0,[ 0 10975 7454 104 742 3574 433 48...,[ 0 10975 7454 104 742 3574 433 48...,[ 0 10975 7454 104 742 3574 433 17...,Social media platform should not be regulated ...,0.966574,[CLS] Social media platforms should not be reg...,[CLS] Social media platform should not be regu...,-1,Social media platforms should be regulated by ...,0.818392


In [17]:
result_df.groupby(by=['topic', 'stance', 'key_point']).size()

topic                                           stance  key_point                                                                                                  
Routine child vaccinations should be mandatory  -1      Child vaccination should not be mandatory as there are often side effects from them                            49
                                                        Children should not be vaccinated because they can have serious side effects                                   55
                                                        It shouldn t be mandatory   every parent should be able to decide whether to vaccinate their childs or not     29
                                                        Parents should be allowed to choose if their child is vaccinated or not                                        27
                                                        Routine child vaccinations should not be mandatory because children may not bear the side effects of

In [38]:
# get the top 10 most frequent KPs 

def get_topic_kps_counts(df):
    unique_topics = df['topic'].unique()
    unique_stances = [-1, 1]
    
    topics = []
    stances = []
    kps = []
    counts = []
    q_scores = []
    args = []
    for t in unique_topics:
        for stance in unique_stances:
            f_df = df[df['topic'] == t]
            f_df = f_df[f_df['stance'] == stance]   
            unique_kps = f_df['key_point'].unique()
           
            for kp in unique_kps:
                topics.append(t)
                stances.append(stance)
                kps.append(kp)
                #q_scores.append(f_df[f_df['key_point'] == kp].iloc[0]['kp_q_score'])
                counts.append(len(f_df[f_df['key_point'] == kp]))
                #args.append(list(f_df[f_df['key_point'] == kp]['arg_id']))

    ret_df = pd.DataFrame({
            "topic": topics,
            "stance": stances,
            "key_point": kps,
            #"q_score": q_scores,
            "counts": counts,
            #"args": args
        })
    return ret_df
    

In [25]:
counts_df = get_topic_kps_counts(result_df)

In [26]:
len(counts_df)

97

In [27]:
# add word counts
counts_df['kp_word_count'] = counts_df['key_point'].str.split().str.len()

In [28]:
counts_df.head(1)

,topic,stance,key_point,q_score,counts,args,kp_word_count
0,Social media platforms should be regulated by ...,-1,Social media platform should not be regulated ...,0.966574,68,"[arg_1_41, arg_1_83, arg_1_96, arg_1_16, arg_1...",16


In [29]:
# sort them 
counts_df = counts_df.sort_values(by=["counts", "q_score", "kp_word_count"], ascending=(False, False, True)).reset_index(drop=True)

In [111]:
import difflib
# get top kps, remove redundant by checking args overlap ratio


FILTER_OVERLAP_THRESHOLD = 0.7

top_filtered_sents = set()

for index, row in counts_df.iterrows():
    s1 = row['key_point']
    if len(top_filtered_sents) == 0:
        top_filtered_sents.add(s1)
        continue
    
    args_1 = list(row['args'])
    scores = []
    for s in top_filtered_sents:
        s_row = counts_df[counts_df['key_point'] == s].iloc[0]
        args_2 = list(s_row['args'])
        scores.append(difflib.SequenceMatcher(None,args_1,args_2).ratio())
    if max(scores) < FILTER_OVERLAP_THRESHOLD:
        top_filtered_sents.add(s1)
    

top_filtered_df = pd.DataFrame({"key_point": list(top_filtered_sents)})


In [112]:
top_filtered_df = pd.merge(top_filtered_df, counts_df, on='key_point')

In [113]:
len(top_filtered_df)

46

In [119]:
counst_df = top_filtered_df.groupby(by=["topic", "stance"]).apply(lambda df: df.sample(frac=1, random_state=42).sort_values(by=["counts", "q_score", "kp_word_count"], ascending=(False, False, True)).head(4)).reset_index(drop=True)

In [120]:
len(counst_df)

21

In [122]:
pd.set_option('display.max_colwidth', None)
counst_df[["topic", "stance", 'key_point', 'q_score', 'kp_word_count', 'counts']].head(24)

,topic,stance,key_point,q_score,kp_word_count,counts
0,Routine child vaccinations should be mandatory,-1,child vaccinations should not be mandatory to avoid any bad effects on them,0.773095,13,58
1,Routine child vaccinations should be mandatory,-1,it is the duty of parents to choose whether or not to vaccinate their children,0.842486,15,30
2,Routine child vaccinations should be mandatory,1,should be mandatory since they are for the health of the child,0.613431,12,117
3,Routine child vaccinations should be mandatory,1,protecting infants must be a priority for all,0.783763,8,69
4,Routine child vaccinations should be mandatory,1,Vaccines should be a mandatory policy so that children do not suffer from these covid problems in the future,0.863401,19,51
5,Routine child vaccinations should be mandatory,1,They should be mandatory because they save millions of children s lives every year,0.903688,14,33
6,Social media platforms should be regulated by the government,-1,Social media platform should not be regulated by the government because it undermines freedom of speech,0.966574,16,68
7,Social media platforms should be regulated by the government,-1,They should not be regulated because they are private companies and that would go against freedom of expression,0.874413,18,36
8,Social media platforms should be regulated by the government,-1,Social media platforms shouldn t be regulated by the government because they can t control everything posted on social media,0.875591,20,34
9,Social media platforms should be regulated by the government,-1,social networks must be free of any system,0.677497,8,27


In [55]:
import pandas as pd
test_ds_labeled_counts = pd.read_csv(r"C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\processed\test_ds_labeled_counts.csv", index_col=None)

In [56]:
test_ds_labeled_counts.head(40)

,topic,stance,key_point,counts
0,Routine child vaccinations should be mandatory,-1,Routine child vaccinations or their side effects are dangerous,53
1,Routine child vaccinations should be mandatory,-1,Mandatory vaccination contradicts basic rights,21
2,Routine child vaccinations should be mandatory,-1,The parents and not the state should decide,21
3,Routine child vaccinations should be mandatory,-1,Routine child vaccinations are not necessary to keep children healthy,11
4,Routine child vaccinations should be mandatory,1,Children should not suffer from preventable diseases,54
5,Routine child vaccinations should be mandatory,1,Routine child vaccinations should be mandatory to prevent virus disease spreading,30
6,Routine child vaccinations should be mandatory,1,Routine child vaccinations are effective,27
7,Routine child vaccinations should be mandatory,1,Routine child vaccinations are necessary to protect others,21
8,Social media platforms should be regulated by the government,-1,Social media regulation harm freedom of speech and other democratic rights,49
9,Social media platforms should be regulated by the government,-1,Social media regulation harms privacy,15


# compare with the ground truth
 now we should compare result_df with test_ds_labeled , each having 500 arg_id

In [47]:
import rouge
# evaluate to ground truth using rouge
evaluator = rouge.Rouge(metrics=['rouge-l'],
                           limit_length=True,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

In [123]:
# comparing test_ds_labeled_counts with counst_df
# for every topic, stance => compare predicted kp with every labeled kp and take max score
import numpy as np
from itertools import product
unique_topics = test_ds_labeled['topic'].unique()
unique_stances = [-1, 1]

topics = []
stances = []
scores = []

for t in unique_topics:
    for s in unique_stances:
        test_df = test_ds_labeled_counts[test_ds_labeled_counts['topic'] == t]
        test_df = test_df[test_df['stance'] == s]
        res_df = counst_df[counst_df['topic'] == t]
        res_df = res_df[res_df['stance'] == s]
        
        unique_predicted_kps = res_df['key_point'].unique()
        unique_labeled_kps = test_df['key_point'].unique()
        kp_scores = []
        for p_kp, l_kp in product(unique_predicted_kps, unique_labeled_kps) :
                score = evaluator.get_scores(p_kp, l_kp)
                kp_scores.append(score['rouge-l']['f'])    

        topics.append(t)
        stances.append(s)
        scores.append(np.mean(kp_scores))
        
    
results_df = pd.DataFrame({"topic": topics, "stance": stances,  "r_scores": scores})
    

In [124]:
results_df.head(6)

,topic,stance,r_scores
0,The USA is a good country to live in,-1,0.147271
1,The USA is a good country to live in,1,0.236621
2,Social media platforms should be regulated by the government,-1,0.282859
3,Social media platforms should be regulated by the government,1,0.218795
4,Routine child vaccinations should be mandatory,-1,0.258473
5,Routine child vaccinations should be mandatory,1,0.176643


In [99]:
# key points in labeled ds are completely new and not another arguments like the way we do it, difficult to get good rouge score with our method.
test_ds_labeled[test_ds_labeled['argument'] == test_ds_labeled[test_ds_labeled['arg_id'] == 'arg_0_99']['key_point'].iloc[0]]

,idx,arg_id,argument,topic,stance,MACE-P,key_point_id,label,key_point,text,sentence1,sentence2,subset
